In [725]:
import pandas as pd
import pyodbc
import datetime
import pathlib
import numpy as np

pd.set_option('display.max_rows', None)

DRIVER = "ODBC Driver 13 for SQL Server"
START_DATE = "12/02/2021"

END_DATE =  "12/08/2021"

LINE = "L1"
MONTH = "12"

START_DATE_ARR = START_DATE.split("/")
END_DATE_ARR = END_DATE.split("/")

FILE_NAME = f"{START_DATE_ARR[1]}_{END_DATE_ARR[1]}{END_DATE_ARR[0]}_{START_DATE_ARR[2]}"
FILE_PATH = pathlib.Path(f"./data/{FILE_NAME}")

In [726]:
sql_conn = pyodbc.connect(f'DRIVER={DRIVER};SERVER=DESKTOP-IE69849;DATABASE=POD_Data_Analysis;Trusted_Connection=yes') 

In [727]:
machine_per_entry_query = """ SELECT [LineNO]
      ,[MachineNO]
      ,[Output]
      ,[OK]
      ,[NG]
      ,[Sampling]
      ,[Cache]
      ,[Other]
      ,[Output_Total]
      ,[OK_Total]
      ,[NG_Total]
      ,[Sampling_Total]
      ,[Cache_Total]
      ,[Other_Total]
      ,[Createtime]
  FROM [POD_Data_Analysis].[dbo].[IoT_POD_MachineOutPutData]
  ORDER BY Createtime
"""

machine_run_data_query = """ SELECT [LineNO]
      ,[MachineNO]
      ,Convert(INT, [TotalInput]) AS [TotalInput]
      ,Convert(INT, [Output]) AS [Output]
      ,Convert(INT, [Sampling]) AS [Sampling]
      ,Convert(INT, [OK]) AS [OK]
      ,Convert(INT, [Others]) AS [Others]
      ,Convert(INT, [NGQty]) AS [NGQty]
      ,[Createtime]
  FROM [POD_Data_Analysis].[dbo].[IoT_POD_MachineRunData]
  ORDER BY Createtime
"""

clear_output_data_query = """ SELECT [LineNO]
      ,[MachineNO]
      ,[TotalInput]
      ,[Output]
      ,[NGQty]
      ,[Sampling]
      ,[OK]
      ,[Others]
      ,[Createtime]
  FROM [POD_Data_Analysis].[dbo].[IoT_POD_MachineClearOutPut]
  ORDER BY Createtime
"""

# machine_per_entry_data_df = pd.read_sql(machine_per_entry_query, sql_conn, parse_dates="Createtime", index_col="Createtime")
# clear_output_data_df = pd.read_sql(clear_output_data_query, sql_conn, parse_dates="Createtime", index_col="Createtime")

machine_run_data_df = pd.read_sql(machine_run_data_query, sql_conn, parse_dates="Createtime", index_col="Createtime")

In [728]:
wc_df_arr = [ machine_run_data_df.loc[(machine_run_data_df["MachineNO"] == x + 1) & (machine_run_data_df["LineNO"].str.contains(LINE))] for x in range(10)]

In [729]:
wc_filtered_arr = [wc_df_arr[x].loc["2021-12-02 00:00:00" : "2021-12-10 23:59:59"] for x in range(10)] 
wc_hourly_arr = [wc_filtered_arr[x].resample("H").last() for x in range(10)]
wc_hourly_arr[1].head()

,LineNO,MachineNO,TotalInput,Output,Sampling,OK,Others,NGQty
Createtime,,,,,,,,
2021-12-02 00:00:00,L1,2,122,122,0,0,1,0
2021-12-02 01:00:00,L1,2,418,412,0,0,0,5
2021-12-02 02:00:00,L1,2,856,848,0,0,1,6
2021-12-02 03:00:00,L1,2,1288,1280,0,0,1,6
2021-12-02 04:00:00,L1,2,1710,1702,0,0,1,6


In [730]:
# conditions = [
#     (wc_hourly_arr[0].Output.sub(wc_hourly_arr[0].lag) > 0),
#     (wc_hourly_arr[0].Output.sub(wc_hourly_arr[0].lag) == 0),
#     (wc_hourly_arr[0].Output.sub(wc_hourly_arr[0].lag) < 0)]
# choices = [(wc_hourly_arr[0].Output.sub(wc_hourly_arr[0].lag)), 0, wc_hourly_arr[0].Output]

# wc_hourly_arr[0]["lag"] = wc_hourly_arr[0].Output.shift(periods=1)
# wc_hourly_arr[0]["HourlyOutput"] = np.select(conditions, choices)
wc_hourly_arr[0].iloc[0]


LineNO        L1 
MachineNO       1
TotalInput    128
Output        128
Sampling        0
OK              0
Others          0
NGQty           0
Name: 2021-12-02 00:00:00, dtype: object

In [731]:
for x in range(10):
    # print(wc_hourly_arr[x].index[x] == wc_hourly_arr[0].index[0])
    # break

    wc_hourly_arr[x]["Input"] = wc_hourly_arr[x][["Output", "Sampling", "OK", "NGQty", "Others"]].sum(axis=1)
    wc_hourly_arr[x]["Yield"] = round(wc_hourly_arr[x]["Output"] / wc_hourly_arr[x]["Input"], 6)
    wc_hourly_arr[x]["YieldWithoutSample"] = round(wc_hourly_arr[x]["Output"] / (wc_hourly_arr[x]["Input"] - wc_hourly_arr[x]["Sampling"]), 6)
    wc_hourly_arr[x]["NGRate"] = round(wc_hourly_arr[x]["NGQty"] / wc_hourly_arr[x]["Input"], 6)
    wc_hourly_arr[x]["lag"] = wc_hourly_arr[x].Output.shift(periods=1)

    conditions = [
        (wc_hourly_arr[x].Output.sub(wc_hourly_arr[x].lag) > 0),
        (wc_hourly_arr[x].Output.sub(wc_hourly_arr[x].lag) == 0),
        (wc_hourly_arr[x].Output.sub(wc_hourly_arr[x].lag) < 0)]

    for i in wc_hourly_arr[x].index:
        if (i == wc_hourly_arr[0].index[0]):
            conditions.append(True)
            
    choices = [(wc_hourly_arr[x].Output.sub(wc_hourly_arr[x].lag)), 0, wc_hourly_arr[x].Output, wc_hourly_arr[x].Output]
    wc_hourly_arr[x]["HourlyOutput"] = np.select(conditions, choices)

In [732]:
wc_hourly_arr[5].head()

,LineNO,MachineNO,TotalInput,Output,Sampling,OK,Others,NGQty,Input,Yield,YieldWithoutSample,NGRate,lag,HourlyOutput
Createtime,,,,,,,,,,,,,,
2021-12-02 00:00:00,L1,6,72,72,0,0,0,0,72,1.0,1.0,0.0,NaN,72.0
2021-12-02 01:00:00,L1,6,338,338,0,0,0,0,338,1.0,1.0,0.0,72.0,266.0
2021-12-02 02:00:00,L1,6,770,770,0,0,0,0,770,1.0,1.0,0.0,338.0,432.0
2021-12-02 03:00:00,L1,6,1144,1144,0,0,0,0,1144,1.0,1.0,0.0,770.0,374.0
2021-12-02 04:00:00,L1,6,1538,1538,0,0,0,0,1538,1.0,1.0,0.0,1144.0,394.0


In [733]:
wc_report = [wc_hourly_arr[x][["LineNO", "MachineNO", "Input", "Output", "Sampling", "OK", "NGQty", "Others", "NGRate", "Yield", "YieldWithoutSample", "HourlyOutput"]] for x in range(10)]

In [786]:
wc_report[0].head()

,LineNO,MachineNO,Input,Output,Sampling,OK,NGQty,Others,NGRate,Yield,YieldWithoutSample,HourlyOutput
Createtime,,,,,,,,,,,,
2021-12-02 00:00:00,L1,1,128,128,0,0,0,0,0.0,1.0,1.0,128.0
2021-12-02 01:00:00,L1,1,414,414,0,0,0,0,0.0,1.0,1.0,286.0
2021-12-02 02:00:00,L1,1,856,856,0,0,0,0,0.0,1.0,1.0,442.0
2021-12-02 03:00:00,L1,1,1286,1286,0,0,0,0,0.0,1.0,1.0,430.0
2021-12-02 04:00:00,L1,1,1702,1702,0,0,0,0,0.0,1.0,1.0,416.0


In [735]:
if FILE_PATH.exists():
    with pd.ExcelWriter(f"{FILE_PATH}/{LINE}_{FILE_NAME}.xlsx") as writer:  
        for x in range(10):
            wc_report[x].to_excel(writer, sheet_name=f'WorkCell_{x + 1}', index=True)
else:
    FILE_PATH.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(f"{FILE_PATH}/{LINE}_{FILE_NAME}.xlsx") as writer:  
        for x in range(10):
            wc_report[x].to_excel(writer, sheet_name=f'WorkCell_{x + 1}', index=True)

In [800]:
import os
from glob import glob

files_name = [y for x in os.walk('./data') for y in glob(os.path.join(x[0], '*.xlsx'))]
df_arr = {}
for x in files_name:
    arr = []
    for y in range(10):
        arr.append(pd.read_excel(x, sheet_name=y, parse_dates=["Createtime"], index_col="Createtime"))
    line_name = x[20:22]
    df_arr[line_name] = arr

In [801]:
df_arr['L1'][0].head()

,LineNO,MachineNO,Input,Output,Sampling,OK,NGQty,Others,NGRate,Yield,YieldWithoutSample,HourlyOutput
Createtime,,,,,,,,,,,,
2021-12-02 00:00:00,L1,1,128,128,0,0,0,0,0,1,1,128
2021-12-02 01:00:00,L1,1,414,414,0,0,0,0,0,1,1,286
2021-12-02 02:00:00,L1,1,856,856,0,0,0,0,0,1,1,442
2021-12-02 03:00:00,L1,1,1286,1286,0,0,0,0,0,1,1,430
2021-12-02 04:00:00,L1,1,1702,1702,0,0,0,0,0,1,1,416


In [803]:
lines_name = [ f[20:22] for f in files_name]
date_arr = list(set([x.date() for x in df_arr[lines_name[0]][0].index]))

In [804]:
date_arr.sort()
date_str_arr = [d.strftime("%m/%d/%Y")  for d in date_arr]

In [807]:
df_arr['L1'][1].loc['12/02/2021'].head()

,LineNO,MachineNO,Input,Output,Sampling,OK,NGQty,Others,NGRate,Yield,YieldWithoutSample,HourlyOutput
Createtime,,,,,,,,,,,,
2021-12-02 00:00:00,L1,2,123,122,0,0,0,1,0.000000,0.991870,0.991870,122
2021-12-02 01:00:00,L1,2,417,412,0,0,5,0,0.011990,0.988010,0.988010,290
2021-12-02 02:00:00,L1,2,855,848,0,0,6,1,0.007018,0.991813,0.991813,436
2021-12-02 03:00:00,L1,2,1287,1280,0,0,6,1,0.004662,0.994561,0.994561,432
2021-12-02 04:00:00,L1,2,1709,1702,0,0,6,1,0.003511,0.995904,0.995904,422


In [808]:
df_arr[lines_name[0]][0].loc[date_str_arr[0]].columns

Index(['LineNO', 'MachineNO', 'Input', 'Output', 'Sampling', 'OK', 'NGQty',
       'Others', 'NGRate', 'Yield', 'YieldWithoutSample', 'HourlyOutput'],
      dtype='object')

In [811]:
time = df_arr[lines_name[0]][0].loc[date_str_arr[0]].index
col = df_arr[lines_name[0]][0].loc[date_str_arr[0]].columns

In [821]:
df = df_arr[lines_name[0]][0].loc[date_str_arr[0]]
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'new column name'})
df

,Createtime,LineNO,MachineNO,Input,Output,Sampling,OK,NGQty,Others,NGRate,Yield,YieldWithoutSample,HourlyOutput
0,2021-12-02 00:00:00,L1,1,128,128,0,0,0,0,0,1,1,128
1,2021-12-02 01:00:00,L1,1,414,414,0,0,0,0,0,1,1,286
2,2021-12-02 02:00:00,L1,1,856,856,0,0,0,0,0,1,1,442
3,2021-12-02 03:00:00,L1,1,1286,1286,0,0,0,0,0,1,1,430
4,2021-12-02 04:00:00,L1,1,1702,1702,0,0,0,0,0,1,1,416
5,2021-12-02 05:00:00,L1,1,2026,2026,0,0,0,0,0,1,1,324
6,2021-12-02 06:00:00,L1,1,2450,2450,0,0,0,0,0,1,1,424
7,2021-12-02 07:00:00,L1,1,2910,2910,0,0,0,0,0,1,1,460
8,2021-12-02 08:00:00,L1,1,3218,3218,0,0,0,0,0,1,1,308
9,2021-12-02 09:00:00,L1,1,3500,3500,0,0,0,0,0,1,1,282
